In [14]:
from functools import lru_cache
import sys
sys.setrecursionlimit(10**9)
# input = sys.stdin.readline
from decimal import Decimal
from functools import cmp_to_key
from collections import Counter
from itertools import permutations
from itertools import combinations
import math
import heapq
from collections import OrderedDict
import bisect
from collections import defaultdict

from collections import deque

class RollbackUnionFind:
    """
    履歴を持ち、直前の操作を（順に）取り消せるUnionFind
    """

    def __init__(self, n: int):
        self.table = [-1] * n
        self.history = []
        # (継続リーダー番号1, 被統合リーダー番号2, 1統合前table値, 2統合前table値)
        # 無効なuniteの場合、被統合リーダー番号を"-1"とする

    def root(self, x: int):
        tbl = self.table
        while tbl[x] >= 0:
            x = tbl[x]
        return x
    
    def is_same(self, x: int, y: int):
        return self.root(x) == self.root(y)
    
    # グループの要素数が多い方にくっつける
    def unite(self, x: int, y: int):
        r1 = self.root(x)
        r2 = self.root(y)
        if r1 == r2:
            self.history.append((r1, -1, -1, -1))
            return False
        d1 = self.table[r1]
        d2 = self.table[r2]
        if d1 <= d2:
            self.table[r2] = r1
            self.table[r1] += d2
            self.history.append((r1, r2, d1, d2))
        else:
            self.table[r1] = r2
            self.table[r2] += d1
            self.history.append((r2, r1, d2, d1))
        return True
    
    def rollback(self):
        if not self.history:
            return False
        r1, r2, d1, d2 = self.history.pop()
        if r2 == -1:
            return False
        self.table[r1] = d1
        self.table[r2] = d2
        return True
    
    def get_size(self, x: int):
        return -self.table[self.root(x)]
    
def main():
    N = int(input())
    mod = 998244353
    ans = [0] * N
    # UnionFind
    uf = RollbackUnionFind(N)
    for _ in range(N-1):
        p, q = map(lambda x: int(x)-1, input().split())
        uf.unite(p, q)
    # 要素数が少ない方から期待値を更新する，要素数が同じ時は後ろの要素から更新する．
    query = deque(uf.history)
    while query:
        p, q, size_p, size_q = query.pop()
        # qの要素数がpの要素数以下ならqの根の期待値から更新
        inv = pow(-1 * (size_p + size_q), -1, mod)
        # print(p,q,size_p,size_q)
        ans[q] = (ans[p] + (-1) * size_q * inv) % mod
        ans[p] = (ans[p] + (-1) * size_p * inv) % mod
        # print(ans)
    print(*ans)
    
if __name__ == "__main__":
    main()

698771048 698771048 964969543 964969543 133099248
